<a href="https://colab.research.google.com/github/Frenchyy1/test/blob/master/Verbatims.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Projet CDA -----------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
from google.colab import files
uploaded = files.upload()

Saving commentaires_clients_enquete_satisfaction.csv to commentaires_clients_enquete_satisfaction.csv


In [6]:
cltmod = pd.read_pickle('verbats.pkl')
clt=pd.read_csv("commentaires_clients_enquete_satisfaction.csv", sep=';',  encoding='latin-1', error_bad_lines=False)
clt.shape
clt.columns

Index(['ID', 'Code', 'Période', 'Semaine', 'Station', 'Destination',
       'Commentaire positif/négatif', 'Thèmes', 'Sous-thèmes', 'Commentaire',
       'Satisfaction globale', 'Recommandation', 'Intention de revenir',
       'Domaine skiable', 'Préparation du séjour',
       'Expérience pratique en station', 'Expérience plaisir en station',
       'Fin du séjour', 'Développement durable', 'Durée du séjour', 'Fidelité',
       'Nationalité', 'Composition du groupe', 'Type de logement',
       'Niveau de ski', 'Mode d'achat du forfait', 'Avec ou sans problèmes'],
      dtype='object')

In [0]:
# Décrire les variables discrètes
import seaborn as sns
listh=clt.columns.tolist()
listhh2=listh[2:]
lista=listhh2[8:]
for a in lista :
    def graphh(n):
        g=sns.catplot(n, data=clt, kind="count")
        g.set_xticklabels(rotation=90)
        return
    graphh(a)

In [0]:
# lecture fichiers retraités par Spad -----------
#clst = pd.read_csv(pathh + "Clusters.txt", sep='|',  encoding='latin-1', 
#                  error_bad_lines=False)

verb=pd.read_csv(pathh + "Mots transformés2.txt", sep='|',  encoding='latin-1', 
                  error_bad_lines=False)
verb.columns
a=verb.isnull().sum()/len(verb)
a.sort_values(ascending=True)
verb=verb.drop(['Développement durable'], axis=1)
d=verb.dropna()
d.dtypes
d1=d.select_dtypes(exclude=['object']) # var quantis
d2=d.select_dtypes(include=['object'])# var discr

# filtrer les variables discrètes selon leur nb de modalités
dico=dict()
for i in d2.columns :
    l=len(d2[i].unique()) 
    print(i,l)
    dico[i]=l
newDict = {key: value for (key, value) in dico.items() if value < 21}
lvar=list(newDict.keys())
d3=d[lvar]    # var discr filtrées

d4= d[['Thèmes', 'Sous-thèmes']]   # variables textuelles à cruncher
d5= d['Commentaire']

d3=pd.get_dummies(d3) 
df=d1.join(d3)
df.columns

# Prédiction satisfaction -------------------------------------------------
X=df.drop(['Satisfaction globale', "Recommandation", "Code", "ID"], axis=1)
y=df['Satisfaction globale']
y.value_counts()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=50)
rf_regressor.fit(X_train, y_train)
rf_regressor.score(X_test, y_test)

#  Calcul importances
pd.DataFrame({'importance':rf_regressor.feature_importances_,
              'features':X.columns})
feature_importance = pd.DataFrame({"features": X.columns[:], 
    "score": rf_regressor.feature_importances_}).sort_values(by="score", ascending=False)
feature_importance1=feature_importance[feature_importance.score > 0.01]
sns.catplot(x="score",y="features", data=feature_importance1, kind="bar" )

y_pred = rf_regressor.predict(X_test)  # calcul de la prédiction

plt.figure(figsize=(5,5))
ax=sns.heatmap((pd.crosstab(np.around(y_pred), y_test, normalize='index'))*100, cmap="YlGnBu",
            annot=True, cbar=False)
ax.set_xlabel('Prédiction')
ax.set_ylabel('Réel')
plt.show(ax)


# Xgboost ---------------------------------------------------
from xgboost import XGBRegressor
regressor_xgb = XGBRegressor()
regressor_xgb.fit(X_train, y_train)
regressor_xgb.score(X_test, y_test)

#  Calcul importances
pd.DataFrame({'importance':regressor_xgb.feature_importances_,
              'features':X.columns})
feature_importance = pd.DataFrame({"features": X.columns[:], 
    "score": regressor_xgb.feature_importances_}).sort_values(by="score", ascending=False)
feature_importance1=feature_importance[feature_importance.score > 0.01]
sns.catplot(x="score",y="features", data=feature_importance1, kind="bar" )

y_pred = regressor_xgb.predict(X_test)  # calcul de la prédiction

plt.figure(figsize=(5,5))
ax=sns.heatmap((pd.crosstab(np.around(y_pred), y_test, normalize='index'))*100, cmap="YlGnBu",
            annot=True, cbar=False)
ax.set_xlabel('Prédiction')
ax.set_ylabel('Réel')
plt.show(ax)

# lien satisfaction et intention recommande -------------------------------
plt.figure(figsize=(8,8))
#ax=sns.heatmap(round((pd.crosstab(np.around(df['Satisfaction globale']), df["Recommandation"], normalize='index'))*100),
#               cmap="YlGnBu",annot=True, cbar=False)
ax=sns.heatmap((pd.crosstab(np.around(df['Satisfaction globale']), df["Recommandation"])),
               cmap="YlGnBu",annot=True, cbar=False, fmt='g')
ax.set_xlabel('Satisfaction globale')
ax.set_ylabel('Recommandation')
plt.show(ax)

# lien satisfaction et intention recommande -------------------------------
plt.figure(figsize=(8,8))
ax=sns.heatmap(round(pd.crosstab(d["Intention de revenir"], d['Recommandation'], normalize='index')*100),
               cmap="YlGnBu",annot=True, cbar=False, fmt='g')
ax.set_xlabel('Recommandation')
ax.set_ylabel('Intention de revenir')
plt.show(ax)

# Xgboost voir à quoi est liée l'intention de revenir certainement --------------------------
df=pd.get_dummies(df, columns=["Recommandation"])
df=pd.get_dummies(df, columns=["Satisfaction globale"])
df['Revenir']=d["Intention de revenir"]
df.columns

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Revenir_R'] = labelencoder.fit_transform(df['Revenir'])
X=df.drop(['Satisfaction globale', "Code", "ID",
           'Intention de revenir_Certainement', 'Intention de revenir_Non',
       'Intention de revenir_Possible', 'Revenir', 'Revenir_R'], axis=1)
y=df['Revenir_R']

y.value_counts()
d['Intention de revenir'].value_counts()


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

from xgboost import XGBRegressor
regressor_xgb = XGBRegressor()
regressor_xgb.fit(X_train, y_train)
regressor_xgb.score(X_test, y_test)

#  Calcul importances
pd.DataFrame({'importance':regressor_xgb.feature_importances_,
              'features':X.columns})
feature_importance = pd.DataFrame({"features": X.columns[:], 
    "score": regressor_xgb.feature_importances_}).sort_values(by="score", ascending=False)
feature_importance1=feature_importance[feature_importance.score > 0.01]
sns.catplot(x="score",y="features", data=feature_importance1, kind="bar" )

y_pred = regressor_xgb.predict(X_test)  # calcul de la prédiction

plt.figure(figsize=(5,5))
ax=sns.heatmap((pd.crosstab(np.around(y_pred), y_test, normalize='index'))*100, cmap="YlGnBu",
            annot=True, cbar=False)
ax.set_xlabel('Prédiction')
ax.set_ylabel('Réel')
plt.show(ax)
# Suite

# Réseau de neurones sur var précédentes
# Croiser avec comportement réel

# Réseau neurones -----------------------------------------
d6= d[['Intention de revenir', 'Nationalité']]
d6=d6[d6['Nationalité']=="Français"]
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
d6['Revenir_R'] = labelencoder.fit_transform(d6['Intention de revenir'])
d6= d6[['Revenir_R']]
d5= d['Commentaire']
df=d6.join(d5)

df.shape
df.to_pickle(pathh + 'verbats.pkl')

from fastai.text import *
mask = np.random.rand(len(df)) < 0.8
df_train = df[mask]
df_test = df[~mask]
df.columns
# Classifier model data
data_lm = TextLMDataBunch.from_df(Path(),
                                  train_df = df_train,
                                  valid_df = df_test,
                                  text_cols = 'Commentaires', 
                                  label_cols= "Revenir_R",
                                  tokenizer= Tokenizer(lang="fr")
                                  )
data_clas = TextClasDataBunch.from_df(pathh, 'texts.csv', vocab=data_lm.train_ds.vocab, bs=32)